In [1]:
!pip install hqq==0.1.8
!pip install transformers==4.46.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━

https://huggingface.co/mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/blob/main/README.md

In [ ]:
import vllm

In [ ]:
import transformers
from threading import Thread

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'
#Load the model
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model     = HQQModelForCausalLM.from_quantized(model_id)

#Optional: set backend/compile
#You will need to install CUDA kernels apriori
# git clone https://github.com/mobiusml/hqq/
# cd hqq/kernels && python setup_cuda.py install
from hqq.core.quantize import *
HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)


def chat_processor(chat, max_new_tokens=1, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    generate_params = dict(
        tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to('cuda'),
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=0.90,
        top_k=50,
        temperature= 0.6,
        num_beams=1,
        repetition_penalty=1.2,
    )

    t = Thread(target=model.generate, kwargs=generate_params)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        print(text, end="", flush=True)

    return outputs

################################################################################################
#Generation
outputs = chat_processor("How do I build a car?", max_new_tokens=1, do_sample=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.23k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

qmodel.pt:   0%|          | 0.00/18.3G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/hqq/models/base.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=m

In [1]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [ ]:
import transformers
from threading import Thread
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'
#Load the model
gc.collect()
torch.cuda.empty_cache()
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
gc.collect()
torch.cuda.empty_cache()
model     = HQQModelForCausalLM.from_quantized(model_id)

gc.collect()
torch.cuda.empty_cache()
#Optional: set backend/compile
#You will need to install CUDA kernels apriori
# git clone https://github.com/mobiusml/hqq/
# cd hqq/kernels && python setup_cuda.py install
from hqq.core.quantize import *
HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)


gc.collect()
torch.cuda.empty_cache()
def chat_processor(chat, max_new_tokens=1, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    generate_params = dict(
        tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to('cuda'),
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=0.90,
        top_k=50,
        temperature= 0.6,
        num_beams=1,
        repetition_penalty=1.2,
    )


    t = Thread(target=model.generate, kwargs=generate_params)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        print(text, end="", flush=True)

gc.collect()
torch.cuda.empty_cache()
return outputs

################################################################################################
#Generation

gc.collect()
torch.cuda.empty_cache()
outputs = chat_processor("How do I build a car?", max_new_tokens=1, do_sample=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/hqq/models/base.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=m

In [ ]:
import transformers
from threading import Thread
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = HQQModelForCausalLM.from_quantized(model_id)

# Optional backend settings
from hqq.core.quantize import HQQLinear, HQQBackend
HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)

def chat_processor(chat, max_new_tokens=1, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(
        tokenizer,
        timeout=10.0,
        skip_prompt=True,
        skip_special_tokens=True
    )

    # Convert input to CUDA with float16 for memory optimization
    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to('cuda', torch.float16)

    generate_params = {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "streamer": streamer,
        "max_new_tokens": max_new_tokens,
        "do_sample": do_sample,
        "top_p": 0.90,
        "top_k": 50,
        "temperature": 0.6,
        "num_beams": 1,
        "repetition_penalty": 1.2,
    }

    # Use no_grad to reduce memory usage
    outputs = []
    with torch.no_grad():
        t = Thread(target=model.generate, kwargs=generate_params)
        t.start()
        for text in streamer:
            outputs.append(text)
            print(text, end="", flush=True)

    gc.collect()
    torch.cuda.empty_cache()

    return outputs

# Generate text
gc.collect()
torch.cuda.empty_cache()
outputs = chat_processor("How do I build a car?", max_new_tokens=1, do_sample=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/hqq/models/base.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=m

In [1]:
import transformers
from threading import Thread
import gc
import torch
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'

# تحديد الجهاز
device = "cuda" if torch.cuda.is_available() else "cpu"

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id)

# تحميل النموذج مع تفعيل `device_map="auto"`
model = HQQModelForCausalLM.from_quantized(
    model_id,
    device_map="auto"  # توزيع أجزاء النموذج تلقائيًا
)

gc.collect()
torch.cuda.empty_cache()

# تحسين التوليد عبر `torch.inference_mode()`
def chat_processor(chat, max_new_tokens=1, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(
        tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
    )

    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to(device)

    with torch.inference_mode():  # يقلل استهلاك الذاكرة
        outputs = model.generate(
            **inputs,
            streamer=streamer,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=0.90,
            top_k=50,
            temperature=0.6,
            num_beams=1,
            repetition_penalty=1.2,
        )

    for text in streamer:
        print(text, end="", flush=True)

    return outputs

gc.collect()
torch.cuda.empty_cache()

# تجربة التوليد
outputs = chat_processor("How do I build a car?", max_new_tokens=1, do_sample=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


TypeError: HQQWrapper.from_quantized() got an unexpected keyword argument 'device_map'

In [ ]:
device_map="auto"

In [ ]:
model = HQQModelForCausalLM.from_quantized(
    model_id,
    device=device,
    device_map=device_map
)

In [2]:
import transformers
from threading import Thread
import gc
import torch
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

gc.collect()
torch.cuda.empty_cache()
device_map="auto"
model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'

# تحديد الجهاز
device = "cuda" if torch.cuda.is_available() else "cpu"

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id)

# تحميل النموذج على CPU أولًا
#model = HQQModelForCausalLM.from_quantized(model_id)
model = HQQModelForCausalLM.from_quantized(
    model_id,
    device_map=device_map
)

# نقل النموذج إلى GPU إذا كان متاحًا
model.to(device)

gc.collect()
torch.cuda.empty_cache()

# تحسين التوليد عبر `torch.inference_mode()`
def chat_processor(chat, max_new_tokens=1, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(
        tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
    )

    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to(device)

    with torch.inference_mode():  # يقلل استهلاك الذاكرة
        outputs = model.generate(
            **inputs,
            streamer=streamer,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=0.90,
            top_k=50,
            temperature=0.6,
            num_beams=1,
            repetition_penalty=1.2,
        )

    for text in streamer:
        print(text, end="", flush=True)

    return outputs

gc.collect()
torch.cuda.empty_cache()

# تجربة التوليد
outputs = chat_processor("How do I build a car?", max_new_tokens=1, do_sample=False)


TypeError: HQQWrapper.from_quantized() got an unexpected keyword argument 'device_map'

In [4]:
import transformers
from threading import Thread
import gc
import torch
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'

# التأكد من أن لدينا GPU متاح
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id)

# محاولة تحميل النموذج بتقليل استهلاك VRAM
model = HQQModelForCausalLM.from_quantized(
    model_id,
    #torch_dtype=torch.float16,  # تقليل حجم النموذج في الذاكرة
    device="meta"  # تحميل بشكل مبدئي إلى meta لتجنب كراش
)

# نقل أجزاء محددة من النموذج إلى GPU تدريجيًا
model = model.to(device)

gc.collect()
torch.cuda.empty_cache()

# تحسين التوليد باستخدام `torch.inference_mode()`
def chat_processor(chat, max_new_tokens=10, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(
        tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
    )

    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            streamer=streamer,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=0.90,
            top_k=50,
            temperature=0.6,
            num_beams=1,
            repetition_penalty=1.2,
        )

    for text in streamer:
        print(text, end="", flush=True)

    return outputs

gc.collect()
torch.cuda.empty_cache()

# تجربة التوليد
outputs = chat_processor("How do I build a car?", max_new_tokens=50, do_sample=False)


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/hqq/models/base.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=m

RuntimeError: Attempted to call `variable.set_data(tensor)`, but `variable` and `tensor` have incompatible tensor type.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, HQQConfig
from hqq.core.quantize import *

# model_id  = "mistral-community/Mixtral-8x22B-v0.1"
model_id  = "mistralai/Mixtral-8x7B-v0.1"

attn_params     = BaseQuantizeConfig(nbits=4, group_size=64, quant_zero=True, quant_scale=True)
attn_params['scale_quant_params']['group_size'] = 256
experts_params = BaseQuantizeConfig(nbits=2, group_size=16, quant_zero=True, quant_scale=True)

#Each type of linear layer (referred to as linear tag) will use different quantization parameters
quant_config  = HQQConfig({
  'self_attn.q_proj': attn_params,
  'self_attn.k_proj': attn_params,
  'self_attn.v_proj': attn_params,
  'self_attn.o_proj': attn_params,

  'block_sparse_moe.experts.w1': experts_params,
  'block_sparse_moe.experts.w2': experts_params,
  'block_sparse_moe.experts.w3': experts_params
})

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="cuda", quantization_config=quant_config)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, HQQConfig
from hqq.core.quantize import *






model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="cuda", quantization_config=quant_config)





In [6]:
import transformers
from threading import Thread
from transformers import AutoModelForCausalLM, AutoTokenizer
from hqq.core.quantize import *
import torch, os

#cache_path    = '.'
compute_dtype = torch.bfloat16
device        = 'auto'
model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'
#Load the model
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device="auto")

#Optional: set backend/compile
#You will need to install CUDA kernels apriori
# git clone https://github.com/mobiusml/hqq/
# cd hqq/kernels && python setup_cuda.py install
from hqq.core.quantize import *
HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)


def chat_processor(chat, max_new_tokens=1, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    generate_params = dict(
        tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to('cuda'),
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=0.90,
        top_k=50,
        temperature= 0.6,
        num_beams=1,
        repetition_penalty=1.2,
    )

    t = Thread(target=model.generate, kwargs=generate_params)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        print(text, end="", flush=True)

    return outputs

################################################################################################
#Generation
outputs = chat_processor("How do I build a car?", max_new_tokens=1, do_sample=False)


OSError: mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

In [1]:
import transformers
from threading import Thread
import torch, os

#cache_path    = '.'
#compute_dtype = torch.bfloat16
#device        = 'auto'
model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'
#Load the model
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model     = HQQModelForCausalLM.from_quantized(model_id, compute_dtype = torch.bfloat16, device='auto')

#Optional: set backend/compile
#You will need to install CUDA kernels apriori
# git clone https://github.com/mobiusml/hqq/
# cd hqq/kernels && python setup_cuda.py install
from hqq.core.quantize import *
HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)


def chat_processor(chat, max_new_tokens=100, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    generate_params = dict(
        tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to('cuda'),
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=0.90,
        top_k=50,
        temperature= 0.6,
        num_beams=1,
        repetition_penalty=1.2,
    )

    t = Thread(target=model.generate, kwargs=generate_params)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        print(text, end="", flush=True)

    return outputs

################################################################################################
#Generation
outputs = chat_processor("How do I build a car?", max_new_tokens=1000, do_sample=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/hqq/models/base.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=m

RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, maia, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone device type at start of device string: auto

In [ ]:
import transformers
from threading import Thread
import torch, os
import gc
#cache_path    = '.'
#compute_dtype = torch.bfloat16
#device        = 'auto'
gc.collect()
torch.cuda.empty_cache()
model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'
#Load the model
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(model_id)
gc.collect()
torch.cuda.empty_cache()
model=HQQModelForCausalLM.from_quantized(model_id, compute_dtype = torch.bfloat16, device='cuda')

#Optional: set backend/compile
#You will need to install CUDA kernels apriori
# git clone https://github.com/mobiusml/hqq/
# cd hqq/kernels && python setup_cuda.py install
from hqq.core.quantize import *
HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)


def chat_processor(chat, max_new_tokens=100, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    generate_params = dict(
        tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to('cuda'),
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=0.90,
        top_k=50,
        temperature= 0.6,
        num_beams=1,
        repetition_penalty=1.2,
    )

    t = Thread(target=model.generate, kwargs=generate_params)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        print(text, end="", flush=True)

    return outputs

################################################################################################
#Generation
outputs = chat_processor("How do I build a car?", max_new_tokens=1000, do_sample=False)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

https://github.com/huggingface/transformers/pull/29637

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [ ]:
import transformers
from threading import Thread
import torch, os
import gc
gc.collect()
torch.cuda.empty_cache()
#cache_path    = '.'
compute_dtype = torch.bfloat16
device        = 'auto'
gc.collect()
torch.cuda.empty_cache()
model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'
#Load the model
gc.collect()
torch.cuda.empty_cache()
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
gc.collect()
torch.cuda.empty_cache()
model     = HQQModelForCausalLM.from_quantized(model_id)

#Optional: set backend/compile
#You will need to install CUDA kernels apriori
# git clone https://github.com/mobiusml/hqq/
# cd hqq/kernels && python setup_cuda.py install
from hqq.core.quantize import *
HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)


def chat_processor(chat, max_new_tokens=100, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    generate_params = dict(
        tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to('cuda'),
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=0.90,
        top_k=50,
        temperature= 0.6,
        num_beams=1,
        repetition_penalty=1.2,
    )

    t = Thread(target=model.generate, kwargs=generate_params)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        print(text, end="", flush=True)

    return outputs

################################################################################################
#Generation
outputs = chat_processor("How do I build a car?", max_new_tokens=1000, do_sample=False)

In [ ]:
import transformers
from threading import Thread
import gc
import torch
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'

# التأكد من أن لدينا GPU متاح
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id)

# تحميل النموذج بدون أي معاملات إضافية
model = HQQModelForCausalLM.from_quantized(model_id)

# نقل النموذج تدريجيًا إلى GPU
model.to(device)

gc.collect()
torch.cuda.empty_cache()

# تحسين التوليد باستخدام `torch.inference_mode()`
def chat_processor(chat, max_new_tokens=10, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(
        tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
    )

    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            streamer=streamer,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=0.90,
            top_k=50,
            temperature=0.6,
            num_beams=1,
            repetition_penalty=1.2,
        )

    for text in streamer:
        print(text, end="", flush=True)

    return outputs

gc.collect()
torch.cuda.empty_cache()

# تجربة التوليد
outputs = chat_processor("How do I build a car?", max_new_tokens=50, do_sample=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/hqq/models/base.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=m

In [3]:






import transformers
from threading import Thread
import time
from transformers import AutoModelForCausalLM, AutoTokenizer
from hqq.core.quantize import *
model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'
#Load the model
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer




model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
print("Model loaded successfully")
print(model.hf_device_map)

tokenizer = AutoTokenizer.from_pretrained(model_id)
text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt").to("cuda:0")

print("Begin text generation")
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print("Generation successful")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

In [ ]:
import logging
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, BitsAndBytesConfig, HqqConfig

hqq_config  = HqqConfig(
    nbits=1,
    group_size=64,
    quant_zero=False,
    quant_scale=False, axis=0) #axis=0 is used by default


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class SpeechToText:
    """Class for converting audio to text using a pre-trained speech recognition model."""

    def __init__(self, model_id: str = "openai/whisper-large-v3", quant_config=None):
        self.model = None
        self.device = None

        if self.model is None:
            self.load_model(model_id)
        else:
            logging.info("Model already loaded.")

    def load_model(self, model_id: str = "openai/whisper-large-v3", quant_config=None):
        """
        Loads the pre-trained speech recognition model and moves it to the specified device.

        Args:
            model_id (str): Identifier of the pre-trained model to be loaded.
        """
        logging.info("Loading model...")
        model = AutoModelForSpeechSeq2Seq.from_pretrained(
            model_id,
            quantization_config=quant_config,
            low_cpu_mem_usage=True,
            use_safetensors=True,
            attn_implementation="flash_attention_2",
            device_map="auto")

        logging.info("Model loaded successfully.")

        processor = AutoProcessor.from_pretrained(model_id)

        self.processor = processor
        self.model = model

    def __call__(
            self,
            chunk_length_s: int = 30,
            stride_length_s: int = 5,
            audio_path: str = "test.mp3",
            max_new_tokens: int = 128,
            batch_size: int = 100,
            language: str = "turkish"):
        """
        Converts audio to text using the pre-trained speech recognition model.

        Args:
            audio_path (str): Path to the audio file to be transcribed.

        Returns:
            str: Transcribed text from the audio.
        """
        pipe = pipeline(
            "automatic-speech-recognition",
            model=self.model,
            chunk_length_s=chunk_length_s,
            stride_length_s=stride_length_s,
            max_new_tokens=max_new_tokens,
            batch_size=100,
            device_map="auto",
            return_timestamps=True,
            tokenizer=self.processor.tokenizer,
            feature_extractor=self.processor.feature_extractor,
            model_kwargs={"use_flash_attention_2": True},
            generate_kwargs={"language": language},
        )
        logging.info("Transcribing audio...")
        result = pipe(audio_path)
        return result

output = SpeechToText(model_id="distil-whisper/distil-large-v3", quant_config=hqq_config) # or bnb_config
transcript = output(
    audio_path = "testv0.mp3",
    chunk_length_s = 30,
    stride_length_s = 5,
    max_new_tokens = 128,
    batch_size = 100,
    language = "english",
)

In [ ]:
import transformers
import torch
import gc
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

# تنظيف الذاكرة قبل تحميل النموذج
gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'

# التأكد من أن لدينا GPU متاح
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id)

# تحميل النموذج على الـ CPU لتجنب استهلاك كامل VRAM
model = HQQModelForCausalLM.from_quantized(model_id)

# تقليل استهلاك الذاكرة
model.half()  # تقليل حجم الـ Tensors إلى FP16
model.to(device)  # نقل النموذج إلى GPU

gc.collect()
torch.cuda.empty_cache()

# تحسين التوليد باستخدام `torch.inference_mode()`
def chat_processor(chat, max_new_tokens=10, do_sample=True):
    tokenizer.use_default_system_prompt = False

    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=0.90,
            top_k=50,
            temperature=0.6,
            num_beams=1,
            repetition_penalty=1.2,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

gc.collect()
torch.cuda.empty_cache()

# تجربة التوليد
response = chat_processor("How do I build a car?", max_new_tokens=50, do_sample=False)
print(response)


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/hqq/models/base.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=m

In [1]:
import transformers
import torch
import gc
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

# تنظيف الذاكرة قبل تحميل النموذج
gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'

# التأكد من توفر الـ GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id)

# تحميل النموذج مع تقسيمه بين GPU و CPU
model = HQQModelForCausalLM.from_quantized(
    model_id,
    offload_folder="offload",  # تخزين بعض الأجزاء في CPU
    offload_state_dict=True
)

# نقل النموذج إلى GPU جزئيًا فقط
model.half().to(device)

gc.collect()
torch.cuda.empty_cache()

# تحسين التوليد باستخدام `torch.inference_mode()`
def chat_processor(chat, max_new_tokens=10, do_sample=True):
    tokenizer.use_default_system_prompt = False

    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=0.90,
            top_k=50,
            temperature=0.6,
            num_beams=1,
            repetition_penalty=1.2,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# تجربة التوليد
response = chat_processor("How do I build a car?", max_new_tokens=50, do_sample=False)
print(response)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


TypeError: HQQWrapper.from_quantized() got an unexpected keyword argument 'offload_folder'

In [ ]:
model = HQQModelForCausalLM.from_quantized(model_id)
model.to("cpu")

In [ ]:
import transformers
import torch
import gc
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

# تنظيف الذاكرة قبل تحميل النموذج
gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'

# التأكد من توفر الـ GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id)

# تحميل النموذج مع تقسيمه بين GPU و CPU
model = HQQModelForCausalLM.from_quantized(model_id)
model.to("cpu")

# نقل النموذج إلى GPU جزئيًا فقط
model.half().to(device)

gc.collect()
torch.cuda.empty_cache()

# تحسين التوليد باستخدام `torch.inference_mode()`
def chat_processor(chat, max_new_tokens=10, do_sample=True):
    tokenizer.use_default_system_prompt = False

    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=0.90,
            top_k=50,
            temperature=0.6,
            num_beams=1,
            repetition_penalty=1.2,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# تجربة التوليد
response = chat_processor("How do I build a car?", max_new_tokens=50, do_sample=False)
print(response)


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/hqq/models/base.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=m

https://github.com/mobiusml/hqq/issues/34

In [ ]:
import torch
import gc
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

# تنظيف الذاكرة قبل التحميل
gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id)

# تحميل النموذج بالكامل على CPU
model = HQQModelForCausalLM.from_quantized(
    model_id
).to("cpu")  # نقل النموذج إلى CPU فقط

gc.collect()
torch.cuda.empty_cache()

# تجربة التوليد
def chat_processor(chat, max_new_tokens=50, do_sample=False):
    tokenizer.use_default_system_prompt = False

    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to("cpu")

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=0.90,
            top_k=50,
            temperature=0.6,
            num_beams=1,
            repetition_penalty=1.2,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

response = chat_processor("How do I build a car?", max_new_tokens=50)
print(response)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/hqq/models/base.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=m

https://github.com/huggingface/transformers/pull/29637

In [1]:
import torch
import gc
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

# تنظيف الذاكرة قبل التحميل
gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id)

# تحميل النموذج بالكامل على CPU
model = HQQModelForCausalLM.from_quantized(
    model_id,
).half().to("cuda")


gc.collect()
torch.cuda.empty_cache()

# تجربة التوليد
def chat_processor(chat, max_new_tokens=50, do_sample=False):
    tokenizer.use_default_system_prompt = False

    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to("cpu")

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=0.90,
            top_k=50,
            temperature=0.6,
            num_beams=1,
            repetition_penalty=1.2,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

response = chat_processor("How do I build a car?", max_new_tokens=50)
print(response)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


TypeError: HQQWrapper.from_quantized() got an unexpected keyword argument 'offload_folder'

In [ ]:
model = HQQModelForCausalLM.from_quantized(
    model_id,
    offload_folder="offload"
).half().to("cuda")


In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model = HQQModelForCausalLM.from_quantized(model_id).to(accelerator.device)

# لاحقًا عند تشغيل التوليد، يمكنك تنفيذ كل شيء باستخدام الـ accelerator
inputs = tokenizer("<s> [INST] How do I build a car? [/INST] ", return_tensors="pt").to(accelerator.device)
with accelerator.device_placement():
    outputs = model.generate(**inputs, max_new_tokens=50)


In [ ]:
import torch
import gc
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

# تنظيف الذاكرة قبل التحميل
gc.collect()
torch.cuda.empty_cache()

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ'

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id)

from accelerate import Accelerator

accelerator = Accelerator()
model = HQQModelForCausalLM.from_quantized(model_id).to(accelerator.device)

# لاحقًا عند تشغيل التوليد، يمكنك تنفيذ كل شيء باستخدام الـ accelerator
inputs = tokenizer("<s> [INST] How do I build a car? [/INST] ", return_tensors="pt").to(accelerator.device)
with accelerator.device_placement():
    outputs = model.generate(**inputs, max_new_tokens=50)



gc.collect()
torch.cuda.empty_cache()

# تجربة التوليد
def chat_processor(chat, max_new_tokens=50, do_sample=False):
    tokenizer.use_default_system_prompt = False

    inputs = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to("cpu")

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=0.90,
            top_k=50,
            temperature=0.6,
            num_beams=1,
            repetition_penalty=1.2,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

response = chat_processor("How do I build a car?", max_new_tokens=50)
print(response)


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/hqq/models/base.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=m

In [7]:
import torch
import os
from transformers import AutoTokenizer
import shutil

# تحديد الجهاز
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")

# تحديد المسارات
model_dir = "mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ"  # مسار النموذج
cache_dir = "/content/cache"  # مسار التخزين المؤقت للبيانات

# التأكد من أن التخزين المؤقت موجود
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

# تحميل الأجزاء من النموذج بشكل جزئي (Sharding)
def load_model_part(part_filename, device=device_cpu):
    """
    تحميل جزء من النموذج إلى الجهاز المحدد (GPU أو CPU)
    """
    part_path = os.path.join(model_dir, part_filename)
    print(f"تحميل {part_filename} إلى {device}")
    model = torch.load(part_path, map_location=device)
    return model

# تقسيم النموذج إلى أجزاء
model_parts = ["/content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt"]  # استبدل هذه بأجزاء النموذج الفعلية

# تحميل الأجزاء على الـ GPU أو الـ CPU
model_parts_on_gpu = []
for part in model_parts:
    part_on_device = load_model_part(part, device=device_gpu)  # تحميل الجزء على الـ GPU
    model_parts_on_gpu.append(part_on_device)

# تقسيم النموذج بين الذاكرة (RAM) و GPU لتوزيع الحمل
def distribute_model_across_memory():
    """
    توزيع النموذج بين GPU و CPU باستخدام تقنيات load sharding.
    """
    model_part_cpu = model_parts_on_gpu[0].to(device_cpu)  # جزء من النموذج في الـ CPU
    model_part_gpu = model_parts_on_gpu[1].to(device_gpu)  # جزء آخر في الـ GPU
    return model_part_cpu, model_part_gpu

# تنفيذ تقسيم النموذج بين الذاكرة
model_part_cpu, model_part_gpu = distribute_model_across_memory()

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained("path_to_tokenizer")  # استبدل بـ مسار التوكنيزر

# مثال لتخزين البيانات المؤقتة
def cache_data(data, filename="cached_data.pth"):
    """
    حفظ البيانات إلى التخزين المؤقت لتحميلها لاحقًا
    """
    cache_path = os.path.join(cache_dir, filename)
    torch.save(data, cache_path)
    print(f"تم حفظ البيانات إلى {cache_path}")

# حفظ جزء من النموذج في التخزين المؤقت
cache_data(model_part_cpu, "part_cpu.pth")
cache_data(model_part_gpu, "part_gpu.pth")

# استخراج بيانات للتدريب أو استخدام النموذج
def generate_output(input_data, model_part_cpu, model_part_gpu):
    """
    توليد مخرجات من النموذج باستخدام أجزاءه المحملة على GPU و CPU.
    """
    input_ids = tokenizer(input_data, return_tensors="pt").to(device_gpu)  # نقل البيانات إلى GPU

    # عمليات على الـ GPU
    output_gpu = model_part_gpu(input_ids)

    # العمليات الأخرى على الـ CPU
    output_cpu = model_part_cpu(input_ids)

    # دمج النتائج من الـ CPU و GPU
    final_output = output_gpu + output_cpu
    return final_output

# مثال على إدخال بيانات وتوليد مخرجات
input_data = "محتوى الإدخال هنا"  # استبدل هذا بالبيانات المناسبة
output = generate_output(input_data, model_part_cpu, model_part_gpu)

# طباعة النتائج
print(f"النتائج: {output}")



تحميل /content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt إلى cuda


<ipython-input-7-117522bccf85>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(part_path, map_location=device)


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 14.12 MiB is free. Process 311539 has 14.72 GiB memory in use. Of the allocated memory 12.91 GiB is allocated by PyTorch, and 1.72 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
import os
from transformers import AutoTokenizer

# تحديد الأجهزة
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")

# تحديد مسارات النموذج والتخزين المؤقت
model_path = "/content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt"
cache_dir = "/content/cache"

# التأكد من وجود التخزين المؤقت
os.makedirs(cache_dir, exist_ok=True)

# تحميل النموذج إلى الجهاز المناسب
def load_model(model_path, device=device_cpu):
    print(f"تحميل النموذج من {model_path} إلى {device}")
    model = torch.load(model_path, map_location=device)
    return model

# تحميل النموذج وتوزيعه بين CPU و GPU
model = load_model(model_path)

# تقسيم النموذج بين الذاكرة الرئيسية والـ GPU
model_cpu, model_gpu = model.to(device_cpu), model.to(device_gpu)

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained("mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ")

# التخزين المؤقت للنموذج لتوفير الذاكرة
def cache_model_part(model_part, filename):
    cache_path = os.path.join(cache_dir, filename)
    torch.save(model_part, cache_path)
    print(f"تم تخزين جزء من النموذج في {cache_path}")

cache_model_part(model_cpu, "model_cpu.pth")
cache_model_part(model_gpu, "model_gpu.pth")

# تنفيذ النموذج على الإدخال
def generate_output(input_text, model_cpu, model_gpu):
    input_ids = tokenizer(input_text, return_tensors="pt").to(device_gpu)
    output_gpu = model_gpu(input_ids)
    output_cpu = model_cpu(input_ids.to(device_cpu))
    return output_gpu + output_cpu

# تجربة الإدخال
test_input = "هذا مثال على الإدخال"
output = generate_output(test_input, model_cpu, model_gpu)

print("النتيجة:", output)


تحميل النموذج من /content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt إلى cpu


<ipython-input-1-52fd269b3a0b>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


In [1]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

# تحديد الأجهزة
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")

# تحديد مسارات النموذج والتخزين المؤقت
model_path = "/content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt"
cache_dir = "/content/cache"

# التأكد من وجود التخزين المؤقت
os.makedirs(cache_dir, exist_ok=True)

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained("mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ")

# تحميل النموذج مع دعم HQQ وتحميل تدريجي
def load_model_hqq(model_path, device=device_cpu):
    print(f"تحميل النموذج من {model_path} إلى {device}")
    model = AutoModelForCausalLM.from_pretrained(
        "mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ",
        cache_dir=cache_dir,
        torch_dtype=torch.float16,
        device_map={"": device}  # تخصيص الأجهزة
    )
    return model

# تحميل النموذج وتوزيعه بين CPU و GPU
model = load_model_hqq(model_path, device_cpu)

# التخزين المؤقت للنموذج لتوفير الذاكرة
def cache_model(model, filename):
    cache_path = os.path.join(cache_dir, filename)
    torch.save(model.state_dict(), cache_path)
    print(f"تم تخزين جزء من النموذج في {cache_path}")

cache_model(model, "model_cached.pth")

# تنفيذ النموذج على الإدخال
def generate_output(input_text, model, tokenizer, device):
    input_ids = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(input_ids.input_ids, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# تجربة الإدخال
test_input = "هذا مثال على الإدخال"
output = generate_output(test_input, model, tokenizer, device_cpu)

print("النتيجة:", output)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


تحميل النموذج من /content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt إلى cpu


config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

OSError: mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

In [2]:
import torch
import os
from transformers import AutoTokenizer
from hqq.engine.hf import HQQModelForCausalLM  # استيراد HQQ مباشرة

# تحديد الأجهزة
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")

# تحديد مسارات النموذج والتخزين المؤقت
model_path = "/content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt"
cache_dir = "/content/cache"

# التأكد من وجود التخزين المؤقت
os.makedirs(cache_dir, exist_ok=True)

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained("mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ")

# تحميل النموذج مع دعم HQQ وتحميل تدريجي
def load_model_hqq(model_path, device=device_cpu):
    print(f"تحميل النموذج من {model_path} إلى {device}")
    model = HQQModelForCausalLM.from_quantized(
        model_path,
        device_map={"": device},  # تخصيص الأجهزة
        dtype=torch.float16
    )
    return model

# تحميل النموذج وتوزيعه بين CPU و GPU
model = load_model_hqq(model_path, device_cpu)

# التخزين المؤقت للنموذج لتوفير الذاكرة
def cache_model(model, filename):
    cache_path = os.path.join(cache_dir, filename)
    torch.save(model.state_dict(), cache_path)
    print(f"تم تخزين جزء من النموذج في {cache_path}")

cache_model(model, "model_cached.pth")

# تنفيذ النموذج على الإدخال
def generate_output(input_text, model, tokenizer, device):
    input_ids = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(input_ids.input_ids, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# تجربة الإدخال
test_input = "هذا مثال على الإدخال"
output = generate_output(test_input, model, tokenizer, device_cpu)

print("النتيجة:", output)


تحميل النموذج من /content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt إلى cpu


TypeError: HQQWrapper.from_quantized() got an unexpected keyword argument 'device_map'

https://huggingface.co/mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/blob/main/README.md

In [3]:
import torch
import os
from transformers import AutoTokenizer
from hqq.engine.hf import HQQModelForCausalLM  # استيراد HQQ مباشرة

# تحديد الأجهزة
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")

# تحديد مسارات النموذج والتخزين المؤقت
model_path = "/content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt"
cache_dir = "/content/cache"

# التأكد من وجود التخزين المؤقت
os.makedirs(cache_dir, exist_ok=True)

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained("mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ")

# تحميل النموذج مع دعم HQQ وتحميل تدريجي
def load_model_hqq(model_path, device=device_cpu):
    print(f"تحميل النموذج من {model_path} إلى {device}")
    model = HQQModelForCausalLM.from_quantized(
        model_path,
        dtype=torch.float16,
        offload_folder=cache_dir  # تخزين مؤقت على القرص
    )
    return model

# تحميل النموذج على CPU أولاً ثم استخدام التخزين المؤقت
model = load_model_hqq(model_path, device_cpu)

# التخزين المؤقت للنموذج لتوفير الذاكرة
def cache_model(model, filename):
    cache_path = os.path.join(cache_dir, filename)
    torch.save(model.state_dict(), cache_path)
    print(f"تم تخزين جزء من النموذج في {cache_path}")

cache_model(model, "model_cached.pth")

# تنفيذ النموذج على الإدخال
def generate_output(input_text, model, tokenizer, device):
    input_ids = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(input_ids.input_ids, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# تجربة الإدخال
test_input = "هذا مثال على الإدخال"
output = generate_output(test_input, model, tokenizer, device_cpu)

print("النتيجة:", output)


تحميل النموذج من /content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt إلى cpu


TypeError: HQQWrapper.from_quantized() got an unexpected keyword argument 'dtype'

In [4]:
import torch
import os
from transformers import AutoTokenizer
from hqq.engine.hf import HQQModelForCausalLM  # استيراد HQQ مباشرة

# تحديد الأجهزة
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")

# تحديد مسارات النموذج والتخزين المؤقت
model_path = "/content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt"
cache_dir = "/content/cache"

# التأكد من وجود التخزين المؤقت
os.makedirs(cache_dir, exist_ok=True)

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained("mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ")

# تحميل النموذج مع دعم HQQ وتحميل تدريجي
def load_model_hqq(model_path, device=device_cpu):
    print(f"تحميل النموذج من {model_path} إلى {device}")
    model = HQQModelForCausalLM.from_quantized(
        model_path,
        #dtype=torch.float16,
        offload_folder=cache_dir  # تخزين مؤقت على القرص
    )
    return model

# تحميل النموذج على CPU أولاً ثم استخدام التخزين المؤقت
model = load_model_hqq(model_path, device_cpu)

# التخزين المؤقت للنموذج لتوفير الذاكرة
def cache_model(model, filename):
    cache_path = os.path.join(cache_dir, filename)
    torch.save(model.state_dict(), cache_path)
    print(f"تم تخزين جزء من النموذج في {cache_path}")

cache_model(model, "model_cached.pth")

# تنفيذ النموذج على الإدخال
def generate_output(input_text, model, tokenizer, device):
    input_ids = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(input_ids.input_ids, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# تجربة الإدخال
test_input = "هذا مثال على الإدخال"
output = generate_output(test_input, model, tokenizer, device_cpu)

print("النتيجة:", output)


تحميل النموذج من /content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt إلى cpu


TypeError: HQQWrapper.from_quantized() got an unexpected keyword argument 'offload_folder'

In [5]:
import torch
import os
from transformers import AutoTokenizer
from hqq.engine.hf import HQQModelForCausalLM  # استيراد HQQ مباشرة

# تحديد الأجهزة
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")

# تحديد مسارات النموذج والتخزين المؤقت
model_path = "/content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt"
cache_dir = "/content/cache"

# التأكد من وجود التخزين المؤقت
os.makedirs(cache_dir, exist_ok=True)

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained("mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ")

# تحميل النموذج مع دعم HQQ وتحميل تدريجي
def load_model_hqq(model_path, device=device_cpu):
    print(f"تحميل النموذج من {model_path} إلى {device}")
    model = HQQModelForCausalLM.from_quantized(
        model_path,
        #dtype=torch.float16,
        #offload_folder=cache_dir  # تخزين مؤقت على القرص
    )
    return model

# تحميل النموذج على CPU أولاً ثم استخدام التخزين المؤقت
model = load_model_hqq(model_path, device_cpu)

# التخزين المؤقت للنموذج لتوفير الذاكرة
def cache_model(model, filename):
    cache_path = os.path.join(cache_dir, filename)
    torch.save(model.state_dict(), cache_path)
    print(f"تم تخزين جزء من النموذج في {cache_path}")

cache_model(model, "model_cached.pth")

# تنفيذ النموذج على الإدخال
def generate_output(input_text, model, tokenizer, device):
    input_ids = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(input_ids.input_ids, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# تجربة الإدخال
test_input = "هذا مثال على الإدخال"
output = generate_output(test_input, model, tokenizer, device_cpu)

print("النتيجة:", output)


تحميل النموذج من /content/models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bit-metaoffload-HQQ/snapshots/459418cf33d7fe46e1da2fd58f0eebdb25f29634/qmodel.pt إلى cpu


Exception: Weight file missing. Check your cache directory.